## Importing libraries

In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import requests
from pandas.io.json import json_normalize
from sklearn.preprocessing import normalize

## Data scraping using BeautifulSoup

In [2]:
soup = BeautifulSoup(urlopen('https://www.makaan.com/price-trends/property-rates-for-buy-in-delhi?page=1'), 'html.parser')
data = []
table = soup.find('table', attrs={'class':'tbl'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if (ele and ele !='-')]) # Get rid of empty values
df=pd.DataFrame(data)
df.head()

,0,1,2,3,4,5
0,Uttam Nagar,"375 - 38,000 / sqft","22,804.91 / sqft",5.4%,See trend,View 3438 properties
1,Dwarka Mor,"444 - 16,667 / sqft","8,154.55 / sqft",See trend,View 946 properties,None
2,Chattarpur,"926 - 2,00,000 / sqft","42,903.63 / sqft",32.2%,See trend,View 236 properties
3,Vasant Kunj,"556 - 1,75,000 / sqft","48,33.87 / sqft",7%,See trend,View 1214 properties
4,Greater kailash 1,"2,805 - 2,00,000 / sqft","74,654.97 / sqft",-15%,See trend,View 64 properties


In [3]:
for i in range(1,26):
    soup=BeautifulSoup(urlopen("https://www.makaan.com/price-trends/property-rates-for-buy-in-delhi?page=%s"%(i+1)))
    data=[]
    table=soup.find("table",attrs={'class':'tbl'})
    table_body=table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if (ele and ele !='-')]) # Get rid of empty values
    df=pd.concat([pd.DataFrame(data),df])

## Cleaning the dataset

In [4]:
df.dropna(inplace=True)

In [5]:
df

,0,1,2,3,4,5
21,Sector 10,"10,769 / sqft","10,769.23 / sqft",-10.7%,See trend,View 1 properties
25,SECTOR 18 DWARKA NEW DELHI,"10,900 / sqft","10,900 / sqft",2.3%,See trend,View 1 properties
56,Sheikh Sarai II,"12,000 / sqft","12,000 / sqft",-4.6%,See trend,View 1 properties
14,Loknayak Puram,"3,111 / sqft","3,111.11 / sqft",-3.7%,See trend,View 1 properties
12,GTB Enclave,"6,000 / sqft","6,000 / sqft",36%,See trend,View 1 properties
...,...,...,...,...,...,...
53,Shivalik,"45,654 - 51,389 / sqft","48,521.4 / sqft",24.8%,See trend,View 2 properties
55,Sector 4 Dwarka,"2,364 - 20,000 / sqft","12,679.11 / sqft",8.4%,See trend,View 364 properties
56,Janakpuri,"1,552 - 21,242 / sqft","16,871.57 / sqft",25%,See trend,View 56 properties
58,Mehrauli,"1,244 - 10,500 / sqft","8,17.04 / sqft",19.6%,See trend,View 367 properties


In [6]:
df.drop([1,3,4,5],axis=1,inplace=True)

In [7]:
df.columns=["locality name","price per sqft"]

In [8]:
df.reset_index(inplace=True,drop=True)

In [9]:
df.head()

,locality name,price per sqft
0,Sector 10,"10,769.23 / sqft"
1,SECTOR 18 DWARKA NEW DELHI,"10,900 / sqft"
2,Sheikh Sarai II,"12,000 / sqft"
3,Loknayak Puram,"3,111.11 / sqft"
4,GTB Enclave,"6,000 / sqft"


## Getting the Latitudes and Longitudes using geopy

In [10]:
geolocator = Nominatim(user_agent="locations",timeout=3)

In [11]:
lat=list()
long=list()
for row in df['locality name']:
    print(row + ", New Delhi")
    location = geolocator.geocode(row + ", New Delhi")
    if(location==None):
        lat.append(np.nan)
        long.append(np.nan)
        continue
    else:
        lat.append(location.latitude)
        long.append(location.longitude)
    

Sector 10, New Delhi
SECTOR 18 DWARKA NEW DELHI, New Delhi
Sheikh Sarai II, New Delhi
Loknayak Puram, New Delhi
GTB Enclave, New Delhi
Jasola Sector 8, New Delhi
Sector 15, New Delhi
Sector 19 Rohini, New Delhi
Sector 19B, New Delhi
Sector-28 Rohini, New Delhi
Zone P II, New Delhi
Kanjhawla, New Delhi
Mayur Vihar 2 Phase, New Delhi
Azad Apartments, New Delhi
east delhi, New Delhi
Sector 21, New Delhi
Sector-18 Rohini, New Delhi
Vikram Vihar, New Delhi
Delhi Cantt, New Delhi
Ambedkar Nagar, New Delhi
Dharampura, New Delhi
Dilshad Colony, New Delhi
Gulmohar Enclave, New Delhi
Mayur Vihar III, New Delhi
Palam Colony, New Delhi
Priyadarshini Vihar, New Delhi
Sector 11 Dwarka Pocket 2, New Delhi
Sector 12, New Delhi
Sector 12 Dwarka New Delhi, New Delhi
Sector 13, New Delhi
Sector 6, New Delhi
Chhawla, New Delhi
Mansarovar garden, New Delhi
Sector 11, New Delhi
SECTOR 7 DWARKA NEW DELHI, New Delhi
Sector 18B Dwarka, New Delhi
B K Dutt Colony, New Delhi
Munirka Vihar, New Delhi
Rana Pratap B

In [12]:
df['latitude']=lat
df['longitude']=long

In [13]:
df.head()

,locality name,price per sqft,latitude,longitude
0,Sector 10,"10,769.23 / sqft",28.570146,77.176045
1,SECTOR 18 DWARKA NEW DELHI,"10,900 / sqft",28.584522,77.046551
2,Sheikh Sarai II,"12,000 / sqft",28.536896,77.222239
3,Loknayak Puram,"3,111.11 / sqft",NaN,NaN
4,GTB Enclave,"6,000 / sqft",NaN,NaN


In [14]:
df.dropna(inplace=True)

In [15]:
df.reset_index(drop=True)

,locality name,price per sqft,latitude,longitude
0,Sector 10,"10,769.23 / sqft",28.570146,77.176045
1,SECTOR 18 DWARKA NEW DELHI,"10,900 / sqft",28.584522,77.046551
2,Sheikh Sarai II,"12,000 / sqft",28.536896,77.222239
3,Sector 15,"14,250 / sqft",39.031714,-90.261223
4,Sector 19B,"11,850.65 / sqft",28.557959,77.058070
...,...,...,...,...
191,Shivalik,"48,521.4 / sqft",28.535375,77.208110
192,Sector 4 Dwarka,"12,679.11 / sqft",28.603668,77.053984
193,Janakpuri,"16,871.57 / sqft",28.621927,77.087476
194,Mehrauli,"8,17.04 / sqft",28.521826,77.178323


## Map of Delhi with locality names superimposed on top

In [16]:
# Map of Delhi using latitude and longitude values
latitude=28.6139
longitude=77.2090
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, location in zip(df['latitude'], df['longitude'], df['locality name']):
    label = '{}'.format(location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi)  
    
map_delhi

## Utilizing the Foursquare API to explore the localities and segment them

In [17]:
CLIENT_ID = '0LFPE41QTZS2HUCWQT24FVGPN02L4OVOSKJU4RE4TGJXO1O3'
CLIENT_SECRET = 'QP1MTTEB0HQY45MUZ2U5EAXEWWQCIIUVABDC4A3FETOUQLDY'
VERSION = '20180605' # Foursquare API version


print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 0LFPE41QTZS2HUCWQT24FVGPN02L4OVOSKJU4RE4TGJXO1O3
CLIENT_SECRET:QP1MTTEB0HQY45MUZ2U5EAXEWWQCIIUVABDC4A3FETOUQLDY


Exploring localities/neighbourhoods of Delhi

In [18]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
delhi_venues = getNearbyVenues(names=df['locality name'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Sector 10
SECTOR 18 DWARKA NEW DELHI
Sheikh Sarai II
Sector 15
Sector 19B
Azad Apartments
east delhi
Sector 21
Vikram Vihar
Delhi Cantt
Ambedkar Nagar
Dharampura
Gulmohar Enclave
Mayur Vihar III
Palam Colony
Priyadarshini Vihar
Sector 12
Sector 13
Sector 6
Chhawla
Mansarovar garden
Sector 11
Sector 18B Dwarka
Munirka Vihar
Rana Pratap Bagh
New Moti Nagar
Commonwealth Games Village
Inderpuri
AKSHARDHAM
KarKardooma
Meera Bagh
West Vinod Nagar
Mandi
Sector 24 Dwarka
Gazipur
IGNOU Road
Safdarjung
Sector 9
Siddharth Enclave
Sainik Nagar
Dwarka 11 Sector
Sector-D Vasant Kunj
Jain Park
Nasirpur
Prashant Vihar
Shaheen Bagh
Ashram
Sector-B Vasant Kunj
West Patel Nagar
Sector 21 Dwarka
Shivaji Marg
Hastsal
Vishnu Garden
Tigri
Sector 5
Deoli Khanpur
Dilshad Garden
Mandawali
Surajmal Vihar
Mandakini Enclave
Rama Park
Masjid moth
Geeta Colony
Mayur Vihar I
Sector 18A Dwarka
Kalyan Vihar
Moti Nagar
Mahipalpur
Sukhdev Vihar
batla house
Azadpur
Vivek Vihar
Connaught Place
North Campus
Sector C Vasant 

In [20]:
delhi_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sector 10,28.570146,77.176045,Delhi Club House,28.572457,77.173489,Comfort Food Restaurant
1,Sector 10,28.570146,77.176045,"STARBUCKS , Sangam Courtyard",28.572560,77.173595,Coffee Shop
2,Sector 10,28.570146,77.176045,Fatty Bao,28.572562,77.173585,Asian Restaurant
3,Sector 10,28.570146,77.176045,"Central Park, Nivedita Kunj",28.568701,77.176501,Park
4,Sector 10,28.570146,77.176045,Ziu,28.572308,77.173699,Thai Restaurant


In [21]:
delhi_venues.shape

(1157, 7)

In [22]:
# one hot encoding
delhi_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")
delhi_onehot.head()

# add neighborhood column back to dataframe
delhi_onehot['Neighborhood'] = delhi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

delhi_onehot.head()

,Volcano,ATM,Accessories Store,Airport,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,...,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [23]:
delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped

,Neighborhood,Volcano,ATM,Accessories Store,Airport,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,...,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant
0,AKSHARDHAM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaknanda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ambedkar Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Anand Lok,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Anand Niketan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,West Vinod Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175,Westend,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176,batla house,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
177,east delhi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
num_top_venues = 5

for hood in delhi_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = delhi_grouped[delhi_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AKSHARDHAM----
                venue  freq
0        Hindu Temple   0.5
1  Athletics & Sports   0.5
2             Volcano   0.0
3              Office   0.0
4         Music Store   0.0


----Alaknanda----
                     venue  freq
0                BBQ Joint   0.2
1        Indian Restaurant   0.2
2               Steakhouse   0.1
3              Pizza Place   0.1
4  New American Restaurant   0.1


----Ambedkar Nagar----
              venue  freq
0  Business Service   1.0
1           Volcano   0.0
2             Plaza   0.0
3       Music Store   0.0
4       Music Venue   0.0


----Anand Lok----
                  venue  freq
0                  Café  0.22
1  Other Great Outdoors  0.11
2         Metro Station  0.11
3                 Hotel  0.11
4           Golf Course  0.11


----Anand Niketan----
         venue  freq
0        Hotel  0.33
1  Pizza Place  0.33
2         Café  0.33
3       Office  0.00
4  Music Venue  0.00


----Anand Vihar----
                     venue  freq
0        

4  American Restaurant  0.12


----New Friends Colony----
           venue  freq
0          Hotel  0.25
1         Lounge  0.25
2  Movie Theater  0.25
3           Café  0.25
4   Optical Shop  0.00


----New Moti Nagar----
                           venue  freq
0  Vegetarian / Vegan Restaurant   1.0
1                         Office   0.0
2                    Music Store   0.0
3                    Music Venue   0.0
4        New American Restaurant   0.0


----Nirman Vihar----
          venue  freq
0     Gift Shop  0.12
1  Liquor Store  0.06
2        Bakery  0.06
3   Pizza Place  0.06
4    Food Court  0.06


----Niti Bagh----
                  venue  freq
0                  Café  0.22
1          Dessert Shop  0.11
2           Pizza Place  0.11
3  Fast Food Restaurant  0.11
4         Metro Station  0.11


----Noida----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.25
1                  Shopping Mall  0.25
2                          Hotel  0.25
3                 

4  Music Store  0.00


----Sector 18B Dwarka----
         venue  freq
0        Hotel  0.25
1         Café  0.25
2          Bar  0.25
3  Coffee Shop  0.25
4  Music Store  0.00


----Sector 19B----
         venue  freq
0         Farm   0.2
1  Pizza Place   0.2
2         Park   0.2
3         Café   0.2
4        Hotel   0.2


----Sector 21----
         venue  freq
0        Hotel  0.25
1         Café  0.25
2          Bar  0.25
3  Coffee Shop  0.25
4  Music Store  0.00


----Sector 21 Dwarka----
         venue  freq
0        Hotel  0.25
1         Café  0.25
2          Bar  0.25
3  Coffee Shop  0.25
4  Music Store  0.00


----Sector 22 Dwarka----
         venue  freq
0         Farm   0.2
1  Pizza Place   0.2
2         Park   0.2
3         Café   0.2
4        Hotel   0.2


----Sector 23 Dwarka----
               venue  freq
0        Pizza Place  0.17
1   Business Service  0.17
2  Recreation Center  0.17
3        Art Gallery  0.17
4      Shopping Mall  0.17


----Sector 4----
                  

          venue  freq
0  Concert Hall   1.0
1       Volcano   0.0
2        Office   0.0
3   Music Store   0.0
4   Music Venue   0.0


----Vishnu Garden----
           venue  freq
0            ATM   0.5
1   Noodle House   0.5
2  Jewelry Store   0.0
3         Museum   0.0
4    Music Store   0.0


----Vivek Vihar----
         venue  freq
0         Bank   1.0
1      Volcano   0.0
2    Multiplex   0.0
3  Music Store   0.0
4  Music Venue   0.0


----West Patel Nagar----
                 venue  freq
0   Light Rail Station  0.14
1                Hotel  0.14
2       Farmers Market  0.14
3                 Park  0.14
4  Sporting Goods Shop  0.14


----West Vinod Nagar----
            venue  freq
0          Garden  0.25
1            Café  0.25
2  Ice Cream Shop  0.25
3  Sandwich Place  0.25
4         Volcano  0.00


----Westend----
                      venue  freq
0               Coffee Shop  0.22
1             Jewelry Store  0.11
2                       Pub  0.11
3        Italian Restaurant  0.1

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = delhi_grouped['Neighborhood']

for ind in np.arange(delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AKSHARDHAM,Athletics & Sports,Hindu Temple,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,Alaknanda,BBQ Joint,Indian Restaurant,Pizza Place,New American Restaurant,Middle Eastern Restaurant,Restaurant,Coffee Shop,Steakhouse,French Restaurant,Food Truck
2,Ambedkar Nagar,Business Service,Vegetarian / Vegan Restaurant,Diner,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
3,Anand Lok,Café,Dessert Shop,Pizza Place,Golf Course,Hotel,Other Great Outdoors,Music Venue,Metro Station,Vegetarian / Vegan Restaurant,Food
4,Anand Niketan,Pizza Place,Café,Hotel,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop


In [27]:
df.head()

,locality name,price per sqft,latitude,longitude
0,Sector 10,"10,769.23 / sqft",28.570146,77.176045
1,SECTOR 18 DWARKA NEW DELHI,"10,900 / sqft",28.584522,77.046551
2,Sheikh Sarai II,"12,000 / sqft",28.536896,77.222239
6,Sector 15,"14,250 / sqft",39.031714,-90.261223
8,Sector 19B,"11,850.65 / sqft",28.557959,77.058070


In [28]:
neighborhoods_venues_sorted=pd.merge(neighborhoods_venues_sorted,df,left_on='Neighborhood',right_on='locality name')

In [29]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,locality name,price per sqft,latitude,longitude
0,AKSHARDHAM,Athletics & Sports,Hindu Temple,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,AKSHARDHAM,"21,603.09 / sqft",28.612517,77.277318
1,Alaknanda,BBQ Joint,Indian Restaurant,Pizza Place,New American Restaurant,Middle Eastern Restaurant,Restaurant,Coffee Shop,Steakhouse,French Restaurant,Food Truck,Alaknanda,"18,636.19 / sqft",28.529336,77.251632
2,Ambedkar Nagar,Business Service,Vegetarian / Vegan Restaurant,Diner,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Ambedkar Nagar,"3,877.55 / sqft",28.519212,77.236210
3,Anand Lok,Café,Dessert Shop,Pizza Place,Golf Course,Hotel,Other Great Outdoors,Music Venue,Metro Station,Vegetarian / Vegan Restaurant,Food,Anand Lok,"6,388.89 / sqft",28.557292,77.219122
4,Anand Niketan,Pizza Place,Café,Hotel,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Anand Niketan,"11,156.62 / sqft",28.578546,77.163865


In [30]:
neighborhoods_venues_sorted.drop(["locality name","latitude","longitude"],axis=1,inplace=True)

In [31]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,price per sqft
0,AKSHARDHAM,Athletics & Sports,Hindu Temple,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,"21,603.09 / sqft"
1,Alaknanda,BBQ Joint,Indian Restaurant,Pizza Place,New American Restaurant,Middle Eastern Restaurant,Restaurant,Coffee Shop,Steakhouse,French Restaurant,Food Truck,"18,636.19 / sqft"
2,Ambedkar Nagar,Business Service,Vegetarian / Vegan Restaurant,Diner,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,"3,877.55 / sqft"
3,Anand Lok,Café,Dessert Shop,Pizza Place,Golf Course,Hotel,Other Great Outdoors,Music Venue,Metro Station,Vegetarian / Vegan Restaurant,Food,"6,388.89 / sqft"
4,Anand Niketan,Pizza Place,Café,Hotel,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,"11,156.62 / sqft"


In [38]:
prices=list()
for price in neighborhoods_venues_sorted['price per sqft']:
    prices.append(price.split()[0])

In [39]:
neighborhoods_venues_sorted['price per sqft']=prices

In [40]:
prices=list()
for price in neighborhoods_venues_sorted['price per sqft']:
    split=price.split(',')
    prices.append(split[0]+split[1])

In [41]:
neighborhoods_venues_sorted['price per sqft']=prices

In [42]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,price per sqft
0,AKSHARDHAM,Athletics & Sports,Hindu Temple,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,21603.09
1,Alaknanda,BBQ Joint,Indian Restaurant,Pizza Place,New American Restaurant,Middle Eastern Restaurant,Restaurant,Coffee Shop,Steakhouse,French Restaurant,Food Truck,18636.19
2,Ambedkar Nagar,Business Service,Vegetarian / Vegan Restaurant,Diner,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,3877.55
3,Anand Lok,Café,Dessert Shop,Pizza Place,Golf Course,Hotel,Other Great Outdoors,Music Venue,Metro Station,Vegetarian / Vegan Restaurant,Food,6388.89
4,Anand Niketan,Pizza Place,Café,Hotel,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,11156.62


In [43]:
delhi_grouped['price per sqft']=neighborhoods_venues_sorted['price per sqft']

In [44]:
delhi_grouped.head()

,Neighborhood,Volcano,ATM,Accessories Store,Airport,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,...,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,price per sqft
0,AKSHARDHAM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21603.09
1,Alaknanda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18636.19
2,Ambedkar Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3877.55
3,Anand Lok,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6388.89
4,Anand Niketan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11156.62


## Clustering neighbourhoods based on its surrounding amenities and average price per sqft of houses (into 5 clusters)

In [45]:
# set number of clusters
kclusters = 5

#Normalizing price per sqft
delhi_grouped_clustering = delhi_grouped.drop('Neighborhood', 1)
delhi_grouped_clustering['price per sqft'] = delhi_grouped_clustering['price per sqft'].astype(float)
delhi_grouped_clustering['price per sqft']=delhi_grouped_clustering['price per sqft']/delhi_grouped_clustering['price per sqft'].max()

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 4, 4, 4, 1, 4, 2, 1])

In [52]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

delhi_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
delhi_merged = pd.merge(delhi_merged,neighborhoods_venues_sorted.set_index('Neighborhood'),left_on="locality name", right_on='Neighborhood')

delhi_merged.head()

,locality name,price per sqft_x,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,price per sqft_y
0,Sector 10,"10,769.23 / sqft",28.570146,77.176045,2,Coffee Shop,Park,Thai Restaurant,Comfort Food Restaurant,Asian Restaurant,Café,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,10769.23
1,SECTOR 18 DWARKA NEW DELHI,"10,900 / sqft",28.584522,77.046551,1,Optical Shop,Indian Restaurant,Light Rail Station,Flower Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,10900
2,Sheikh Sarai II,"12,000 / sqft",28.536896,77.222239,2,Pizza Place,Café,Clothing Store,Electronics Store,Food & Drink Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,12000
3,Sector 19B,"11,850.65 / sqft",28.557959,77.058070,4,Pizza Place,Park,Farm,Hotel,Café,Food,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,11850.65
4,Azad Apartments,"25,000 / sqft",28.633456,77.311515,2,Café,Food Truck,Vegetarian / Vegan Restaurant,Diner,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop,25000


In [53]:
delhi_merged.drop(['price per sqft_x'],axis=1,inplace=True)

In [54]:
delhi_merged.rename(columns={"price per sqft_y": "price per sqft"},inplace=True)

In [55]:
delhi_merged

,locality name,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,price per sqft
0,Sector 10,28.570146,77.176045,2,Coffee Shop,Park,Thai Restaurant,Comfort Food Restaurant,Asian Restaurant,Café,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,10769.23
1,SECTOR 18 DWARKA NEW DELHI,28.584522,77.046551,1,Optical Shop,Indian Restaurant,Light Rail Station,Flower Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,10900
2,Sheikh Sarai II,28.536896,77.222239,2,Pizza Place,Café,Clothing Store,Electronics Store,Food & Drink Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,12000
3,Sector 19B,28.557959,77.058070,4,Pizza Place,Park,Farm,Hotel,Café,Food,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,11850.65
4,Azad Apartments,28.633456,77.311515,2,Café,Food Truck,Vegetarian / Vegan Restaurant,Diner,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop,25000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,Shivalik,28.535375,77.208110,3,Coffee Shop,Café,Donut Shop,Pizza Place,Indian Restaurant,Sandwich Place,Asian Restaurant,Miscellaneous Shop,Chinese Restaurant,French Restaurant,48521.4
175,Sector 4 Dwarka,28.603668,77.053984,1,Electronics Store,Vegetarian / Vegan Restaurant,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,12679.11
176,Janakpuri,28.621927,77.087476,2,Park,Gym / Fitness Center,Café,Bakery,Vegetarian / Vegan Restaurant,Food & Drink Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,16871.57
177,Mehrauli,28.521826,77.178323,1,Thai Restaurant,Restaurant,Market,Flower Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,817.04


## Visulising the clusters on a map

In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(delhi_merged['latitude'], delhi_merged['longitude'], delhi_merged['locality name'], delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [57]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 0, delhi_merged.columns[[0] + list(range(4, delhi_merged.shape[1]))]]

,locality name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,price per sqft
32,Sainik Nagar,ATM,Motorcycle Shop,Furniture / Home Store,IT Services,Café,Vegetarian / Vegan Restaurant,Food & Drink Shop,Garden,Fried Chicken Joint,French Restaurant,15894.74
44,Vishnu Garden,Noodle House,ATM,Vegetarian / Vegan Restaurant,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,737.04
94,Tughlakabad,ATM,Train Station,Light Rail Station,Food & Drink Shop,Gastropub,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,4296.3
95,Sector 17 Dwarka,ATM,Vegetarian / Vegan Restaurant,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,10743.19
98,Mohan Garden,ATM,Indian Restaurant,IT Services,Vegetarian / Vegan Restaurant,Flower Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,10658.05
107,Badarpur,Train Station,ATM,IT Services,Food & Drink Shop,Gastropub,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,8759.71
129,Neb Sarai,ATM,Vegetarian / Vegan Restaurant,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,7212.07
133,Najafgarh,Food & Drink Shop,ATM,Mobile Phone Shop,Vegetarian / Vegan Restaurant,Gastropub,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,6765.43
146,Patel Nagar,ATM,Café,Vegetarian / Vegan Restaurant,Food & Drink Shop,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,24988.3
158,Panchsheel Enclave,ATM,Vegetarian / Vegan Restaurant,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,34222.81


## Cluster 2 

In [58]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 1, delhi_merged.columns[[0] + list(range(4, delhi_merged.shape[1]))]]

,locality name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,price per sqft
1,SECTOR 18 DWARKA NEW DELHI,Optical Shop,Indian Restaurant,Light Rail Station,Flower Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,10900
5,east delhi,Garden,Museum,Vegetarian / Vegan Restaurant,Japanese Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,7578.95
7,Vikram Vihar,Concert Hall,Vegetarian / Vegan Restaurant,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,20454.55
8,Ambedkar Nagar,Business Service,Vegetarian / Vegan Restaurant,Diner,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,3877.55
9,Dharampura,Men's Store,Gift Shop,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,1883.18
...,...,...,...,...,...,...,...,...,...,...,...,...
169,Burari,Paper / Office Supplies Store,Vegetarian / Vegan Restaurant,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,5633.63
170,Bindapur,Bookstore,Fast Food Restaurant,Pizza Place,ATM,Insurance Office,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,647.83
171,Sarita Vihar,Train Station,Juice Bar,Park,Market,Vegetarian / Vegan Restaurant,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,14364.24
175,Sector 4 Dwarka,Electronics Store,Vegetarian / Vegan Restaurant,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,12679.11


## Cluster 3 

In [59]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 2, delhi_merged.columns[[0] + list(range(4, delhi_merged.shape[1]))]]

,locality name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,price per sqft
0,Sector 10,Coffee Shop,Park,Thai Restaurant,Comfort Food Restaurant,Asian Restaurant,Café,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,10769.23
2,Sheikh Sarai II,Pizza Place,Café,Clothing Store,Electronics Store,Food & Drink Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,12000
4,Azad Apartments,Café,Food Truck,Vegetarian / Vegan Restaurant,Diner,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Court,Food & Drink Shop,25000
11,Mayur Vihar III,Shoe Store,Park,Café,Vegetarian / Vegan Restaurant,Food,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,826.74
21,New Moti Nagar,Vegetarian / Vegan Restaurant,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Flower Shop,4896.77
34,Sector-D Vasant Kunj,Vegetarian / Vegan Restaurant,Café,Clothing Store,Pizza Place,Indian Restaurant,Furniture / Home Store,French Restaurant,Food Truck,Food Court,Indie Movie Theater,18715.73
40,Sector-B Vasant Kunj,Vegetarian / Vegan Restaurant,Café,Clothing Store,Pizza Place,Indian Restaurant,Furniture / Home Store,French Restaurant,Food Truck,Food Court,Indie Movie Theater,17267.86
49,Mandawali,Café,Food,Ice Cream Shop,Vegetarian / Vegan Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,1085.86
50,Surajmal Vihar,Burger Joint,Park,Burrito Place,Café,Food Court,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,1737.04
65,Sector C Vasant Kunj,Vegetarian / Vegan Restaurant,Café,Clothing Store,Pizza Place,Indian Restaurant,Furniture / Home Store,French Restaurant,Food Truck,Food Court,Indie Movie Theater,25807.94


## Cluster 4

In [60]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 3, delhi_merged.columns[[0] + list(range(4, delhi_merged.shape[1]))]]

,locality name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,price per sqft
17,Mansarovar garden,Furniture / Home Store,Hotel,Coffee Shop,Vegetarian / Vegan Restaurant,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,86705.2
64,Connaught Place,Indian Restaurant,Café,Bar,Chinese Restaurant,Clothing Store,Coffee Shop,Lounge,Pub,South Indian Restaurant,Deli / Bodega,35892.78
78,South Delhi,Garden,Museum,Vegetarian / Vegan Restaurant,Japanese Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,37468.92
92,Okhla,Indian Restaurant,Playground,Chinese Restaurant,Concert Hall,Coffee Shop,Farm,Farmers Market,Fast Food Restaurant,Fabric Shop,Flea Market,40146.69
109,Delhi,Garden,Museum,Vegetarian / Vegan Restaurant,Japanese Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,46242.13
111,Preet Vihar,Fast Food Restaurant,Pizza Place,Arcade,South Indian Restaurant,Chinese Restaurant,Light Rail Station,BBQ Joint,Ice Cream Shop,Fabric Shop,Electronics Store,39781.04
121,Uday Park,Pizza Place,Music Venue,Fast Food Restaurant,Metro Station,Food & Drink Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,35456.79
126,Shalimar Bagh,Shoe Store,Vegetarian / Vegan Restaurant,Diner,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,34502.07
134,Om Vihar,Train Station,ATM,Indian Restaurant,Mobile Phone Shop,Health & Beauty Service,Food & Drink Shop,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,85855.44
139,Chattarpur,Resort,Indian Restaurant,Pub,Café,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,42903.63


## Cluster 5

In [61]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 4, delhi_merged.columns[[0] + list(range(4, delhi_merged.shape[1]))]]

,locality name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,price per sqft
3,Sector 19B,Pizza Place,Park,Farm,Hotel,Café,Food,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,11850.65
6,Sector 21,Coffee Shop,Bar,Café,Hotel,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,7861.03
15,Sector 13,Hotel,Modern European Restaurant,Café,Vegetarian / Vegan Restaurant,Food & Drink Shop,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,14444.44
18,Sector 18B Dwarka,Coffee Shop,Bar,Café,Hotel,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,9473.59
31,Siddharth Enclave,Hotel,Volcano,Food,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,9726.77
39,Ashram,Hotel,Indian Restaurant,Breakfast Spot,Pizza Place,Sandwich Place,Light Rail Station,Snack Place,Café,Bakery,Food,1245.45
42,Sector 21 Dwarka,Coffee Shop,Bar,Café,Hotel,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,4400
56,Sector 18A Dwarka,Coffee Shop,Bar,Café,Hotel,Food,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,12773.12
57,Kalyan Vihar,Pet Store,Department Store,Food Truck,Hotel,Cheese Shop,Bakery,Vegetarian / Vegan Restaurant,Food & Drink Shop,Furniture / Home Store,Fried Chicken Joint,7764.23
59,Mahipalpur,Hotel,Accessories Store,Indian Restaurant,Plaza,Road,Shoe Store,Bed & Breakfast,Vegetarian / Vegan Restaurant,Food,Fried Chicken Joint,5863.64
